# checking a dbachecks PR

![First Power Bi](..\dbachecks\images\dbachecks-logo.png )

These notebooks are designed to help when there is a new PR to ensure that nothing obvious is broken

## setting up the containers

Of course, this will mean that some checks are not able to be tested so we will have to rely on the other Pester checks for those

In [1]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'

########################################################
Write-Output "Creating Directory $FolderPath"
if(Test-Path $FolderPath){
    Write-Output "Path $FolderPath exists already"
}else {
 $null =    New-Item $FolderPath -ItemType Directory
}

Write-Output "Creating Directory $FolderPath\SQL1"
if(Test-Path "$FolderPath\SQL1"){
    Write-Output "Directory SQL1 exists already"
    Get-ChildItem "$FolderPath\SQL1" -Recurse| Remove-Item -Recurse -Force
}else {
$null = New-Item "$FolderPath\SQL1"-ItemType Directory
}
Write-Output "Creating File $FolderPath\SQL1\dummyfile.txt"
if(Test-Path "$FolderPath\SQL1\dummyfile.txt"){
    Write-Output "dummyfile.txt exists already"
}else {
$null =  New-Item "$FolderPath\SQL1\dummyfile.txt" -ItemType file
}

Write-Output "Creating Directory $FolderPath\SQL2"
if(Test-Path "$FolderPath\SQL2"){
    Write-Output "Directory SQL2 exists already"
    Get-ChildItem "$FolderPath\SQL2" -Recurse| Remove-Item -Recurse -Force
}else {
 $null =    New-Item "$FolderPath\SQL2"-ItemType Directory
}
Write-Output "Creating File $FolderPath\SQL2\dummyfile.txt"
if(Test-Path "$FolderPath\SQL2\dummyfile.txt"){
    Write-Output "dummyfile.txt exists already"
}else {
$null =  New-Item "$FolderPath\SQL2\dummyfile.txt" -ItemType file
}

Write-Output "Creating a credential file for the containers - Please don't do this in production"

$sqladminPassword = ConvertTo-SecureString 'dbatools.IO' -AsPlainText -Force 
$cred = New-Object System.Management.Automation.PSCredential ('sqladmin', $sqladminpassword)
$Cred | Export-Clixml -Path $FolderPath\sqladmin.cred
Write-Output "Credential file created"

Write-Output "Setting the docker-compose files values"
$dockercompose = (Get-Content dockercompose.yml) -replace '__ReplaceME__' , $FolderPath
# $dockercompose
$dockercompose | Set-Content $FolderPath\docker-compose.yml



Creating Directory C:\Users\mrrob\Documents\dbachecks
Path C:\Users\mrrob\Documents\dbachecks exists already
Creating Directory C:\Users\mrrob\Documents\dbachecks\SQL1
Directory SQL1 exists already
Creating File C:\Users\mrrob\Documents\dbachecks\SQL1\dummyfile.txt
Creating Directory C:\Users\mrrob\Documents\dbachecks\SQL2
Directory SQL2 exists already
Creating File C:\Users\mrrob\Documents\dbachecks\SQL2\dummyfile.txt
Creating a credential file for the containers - Please don't do this in production
Credential file created
Setting the docker-compose files values


## Containers

We are going to create two SQL 2019 containers using the sqldbawithabeard\dbachecks [image from the Docker Hub](https://hub.docker.com/repository/docker/sqldbawithabeard/dbachecksdemo). 

The first time it is going to pull the image sqldbawithabeard/dbachecksdemo from the Docker Hub. If you wish to do this first, you can run 

`docker pull sqldbawithabeard/dbachecksdemo`

In [3]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
Set-Location $FolderPath
docker-compose up -d

Creating network "dbachecks_BeardsMagicNetwork" with the default driver
Creating dbachecks_SQL2019-1_1 ... 

Creating dbachecks_SQL2019_1   ... 


Creating dbachecks_SQL2019_1   ... done

Creating dbachecks_SQL2019-1_1 ... done



In [4]:
docker ps -a

CONTAINER ID        IMAGE                            COMMAND                  CREATED             STATUS                            PORTS                                             NAMES
a7e4bc67b6b9        sqldbawithabeard/dbachecksdemo   "/opt/mssql/bin/perm…"   8 seconds ago       Up 7 seconds                      0.0.0.0:5022->5022/tcp, 0.0.0.0:15592->1433/tcp   dbachecks_SQL2019_1
d1c9b65927d2        sqldbawithabeard/dbachecksdemo   "/opt/mssql/bin/perm…"   8 seconds ago       Up 7 seconds                      0.0.0.0:5023->5023/tcp, 0.0.0.0:15593->1433/tcp   dbachecks_SQL2019-1_1
9bc8232d1ea5        sqldbawithabeard/dbachecksdemo   "/opt/mssql/bin/perm…"   7 hours ago         Exited (255) About a minute ago   0.0.0.0:5022->5022/tcp, 0.0.0.0:15592->1433/tcp   agent_SQL2019_1
9d5f1056ecb5        sqldbawithabeard/dbachecksdemo   "/opt/mssql/bin/perm…"   7 hours ago         Exited (255) About a minute ago   0.0.0.0:5023->5023/tcp, 0.0.0.0:15593->1433/tcp   agent_SQL2019-1_1


if required

In [5]:
docker logs dbachecks_SQL2019_1

SQL Server 2019 will run as non-root by default.
This container is running as user mssql.
Your master database file is owned by mssql.
To learn more visit https://go.microsoft.com/fwlink/?linkid=2099216.
2020-05-29 13:48:36.13 Server      Microsoft SQL Server 2019 (RTM) - 15.0.2000.5 (X64) 

	Sep 24 2019 13:48:23 

	Copyright (C) 2019 Microsoft Corporation

	Developer Edition (64-bit) on Linux (Ubuntu 16.04.6 LTS) <X64>

2020-05-29 13:48:36.13 Server      UTC adjustment: 0:00

2020-05-29 13:48:36.13 Server      (c) Microsoft Corporation.

2020-05-29 13:48:36.14 Server      All rights reserved.

2020-05-29 13:48:36.14 Server      Server process ID is 32.

2020-05-29 13:48:36.14 Server      Logging SQL Server messages in file '/var/opt/mssql/log/errorlog'.

2020-05-29 13:48:36.14 Server      Registry startup parameters: 

	 -d /var/opt/mssql/data/master.mdf

	 -l /var/opt/mssql/data/mastlog.ldf

	 -e /var/opt/mssql/log/errorlog

2020-05-29 13:48:36.14 Server      SQL Server detected 1 so

In [ ]:
docker logs dbachecks_SQL2019-1_1